In [ ]:
pip install skimpy


In [ ]:
pip install matplotlib


In [ ]:
pip install seaborn


In [ ]:
pip install scikit-learn


In [ ]:
pip install openpyxl


In [ ]:
os.chdir('/Users/wenjing/Desktop/DSA3101')


In [ ]:
import sys
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from skimpy import skim
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OrdinalEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from collections import Counter
import warnings
from openpyxl import *
from datetime import timedelta
warnings.filterwarnings("ignore")


In [ ]:
data = pd.read_csv('./data/Online_Sales.csv')
data.head()


In [ ]:
skim(data)


In [ ]:
data = data.drop(["Product_SKU", "Product_Description", "Product_Category", "Delivery_Charges", "Coupon_Status"], axis=1)
data["Amount"]=data["Avg_Price"]*data["Quantity"]
data = data.drop(["Quantity", "Avg_Price"], axis=1)
data.head()


In [ ]:
# Join customer tenure information
customer = pd.read_excel('./data/CustomersData.xlsx')
data = pd.merge(data, customer, on="CustomerID", how='left')
data = data.drop(["Gender", "Location"], axis=1)
data.head()


 ## Exploratory Data Analysis

In [ ]:
# Time span of the transactions is 1 year
earliest_data = data['Transaction_Date'].min()
latest_date = data['Transaction_Date'].max()

print(f"earliest: {earliest_data}")
print(f"latest: {latest_date}")


In [ ]:
# Check for duplicate rows
if data['Transaction_ID'].duplicated().any():
    print("id duplicated")
    data = data.drop_duplicates(subset='Transaction_ID', keep='first')
else:
    print("id identical")


In [ ]:
data['Transaction_Date'] = pd.to_datetime(data['Transaction_Date'])


In [ ]:
# Inspect distribution of customer spendings
spending_by_customer = data.groupby('CustomerID')['Amount'].sum()

plt.figure(figsize=(12, 6))
sns.histplot(spending_by_customer, bins=30, kde=True)
plt.title('Distribution of Total Spent by Clients')
plt.xlabel('Total Spent')
plt.ylabel('Number of Clients')
plt.show()

# We can see that the graph is highly skewed to the left
# We can see that most customers spend below $10000


In [ ]:
# Inspect distribution of tenure months
plt.figure(figsize=(12, 6))
sns.histplot(data['Tenure_Months'], bins=range(min(data['Tenure_Months']), max(data['Tenure_Months']) + 1), kde=True)
plt.title('Distribution of Tenure Months')
plt.xlabel('Tenure Months')
plt.ylabel('Frequency')
plt.show()

# No obvious trend in trenure months of customers


In [ ]:
# Inspect distribution of number of transactions
purchase_counts = data.groupby('CustomerID')['Transaction_ID'].nunique()

plt.figure(figsize=(12,6))
sns.histplot(purchase_counts, bins=range(1, purchase_counts.max()+1),kde = True)
plt.title('Distribution of Purchase Counts per Customer')
plt.xlabel('Number of Purchases')
plt.ylabel('Number of Customers')
plt.show()

# We can see that the graph is highly skewed to the left
# Most of our customers purchase below 50 times


 ## Feature engineering

In [ ]:

reference_date = pd.to_datetime("2020-01-01")

grouped = data.groupby('CustomerID').agg(
    join_date=('Transaction_Date', 'min'),
    last_purchase_date=('Transaction_Date', 'max'),
    frequency=('Transaction_ID', 'count'),
    monetary_value=('Amount', 'mean')
).reset_index()

grouped['recency'] = (reference_date - grouped['last_purchase_date']).dt.days
grouped['T'] = (reference_date - grouped['join_date']).dt.days

print(grouped.head())


 ## CLV prediction

In [ ]:
pip install lifetimes


In [ ]:
from lifetimes import BetaGeoFitter, GammaGammaFitter

#penalizer_coef = 0.1

# Fit BG/NBD model (predict purchase frequency)
bgf = BetaGeoFitter(penalizer_coef=penalizer_coef)
bgf.fit(grouped["frequency"], grouped["recency"], grouped["T"], initial_params=initial_params)

# Fit Gamma-Gamma model (predict spending per transaction)
ggf = GammaGammaFitter(penalizer_coef=penalizer_coef)
ggf.fit(grouped["frequency"], grouped["monetary_value"])

# Predict the transaction frequency and average amount in 1 year
grouped['predicted_transactions'] = bgf.predict(365, grouped['frequency'], grouped['recency'], grouped['T'])
grouped['predicted_average_value'] = ggf.conditional_expected_average_profit(grouped['frequency'], grouped['monetary_value'])

# CLV in 1 year
grouped['predicted_1yr_clv'] = grouped['predicted_transactions'] * grouped['predicted_average_value']
print(grouped.head())



In [ ]:
corr_matrix_pearson = grouped[['frequency','monetary_value','recency','T','predicted_1yr_clv']].corr(method = 'pearson')
mask = np.triu(np.ones_like(corr_matrix_pearson, dtype = bool))

fig,ax = plt.subplots(figsize = (10, 5.5))
sns.heatmap(corr_matrix_pearson, 
            annot = True, 
            annot_kws = {'fontsize':5.5, 'fontweight':'bold'},
            fmt = '.3f',
            linewidths = 0.6,
            cmap = 'RdBu_r', 
            mask = mask, 
            square = True,
            ax = ax)
ax.set_xticklabels(ax.get_xticklabels(), rotation = 45, horizontalalignment = 'right')
ax.tick_params(labelsize = 7, labelcolor = 'black')
ax.set_title("Correlation Matrix-Pearson", fontsize = 10, fontweight = 'bold', color = 'black')
fig.show()

# From the correlation matrix, we can see that predicted_1yr_clv is not strongly correlated to other variables
# Thus, a combination of features should be used to segment the customers 


 ## Customer segmentation by Random Forest

In [ ]:
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt

X = grouped[['recency', 'frequency', 'monetary_value', 'T', 'predicted_1yr_clv']]
kmeans = KMeans(n_clusters=5, random_state=42)
grouped['cluster'] = kmeans.fit_predict(X)

grouped['cluster'].value_counts().plot(kind='bar')
plt.show()


In [ ]:
# Visualisation of customer segments distribution
features = X[['recency', 'frequency', 'monetary_value', 'T', 'predicted_1yr_clv']]

fig, axes = plt.subplots(nrows=2, ncols=3, figsize=(15, 10))
fig.suptitle('Distributuion of Features by Cluster')
axes = axes.flatten()

for i, feature in enumerate(features):
    grouped.boxplot(column=feature, by='cluster', ax=axes[i])
    axes[i].set_title(f'{feature} by Cluster')
    axes[i].set_xlabel('Cluster')
    axes[i].set_ylabel(feature)

# Hide the unused subplot (the last one)
axes[-1].axis('off')

plt.show()


In [ ]:
# Inspection of statistics of each cluster
cluster_summary = grouped.groupby('cluster').agg({
    'T':'mean',
    'monetary_value': 'mean',
    'frequency': 'mean',
    'recency': 'mean',
    'predicted_1yr_clv': 'mean'
})
print(cluster_summary)


 ## Explanation of the clusters

 ### Cluster 0: Lost

 #### This group is characterised by the longest time till their last purchase and lowest average frequency of purchase, thus is deemed to be lost.



 ### Cluster 1: Loyal Customers

 #### This group is characterised by the highest average purchase value, relatively recent purchases and high CLV, indicating they are a group of relatively stable and regular customers.



 ### Cluster 2: Potential Loyalties

 #### This group is characterised by relatively high average purchase value and frequency, indicating their potential to upgrade to loyal customers.



 ### Cluster 3: Active customers (highest clv and recency)

 #### This group is characterised by the shortest tenure length and most recent purchases, indicating they are converted recently and actively making purchases.



 ### Cluster 4: At Risk

 #### This group is characterised by relatively long duration since their last purchase, meaning they might be lost without proper maintainance measures.

 ## Maximise Campaign ROI

 ### 1. Budget and resource allocation

 #### Basded on customer previous transactions, banks can predict the cusotmer lifetime value. Thus, banks can allocate the cost of acquiring a customer based on the potential value of acquirng this customer to maximise the overall campaign ROI.



 ### 2. Targeted campaign strategies -- Customer-based strategy

 #### Basded on previous transaction trends, preferences in certain product category and/or responsiveness to campaigns, banks can design more personalized marketing campaigns targeted at each customer. For example, give exclusive welcome offers to active customers, VIP services or early access to new products to loyal customers, and re-engage the at risk customers before they are lost by enticing them with product categories that they are interested in. This can ensure maximised campaign returns, thus ROI.



 ### 3. Identifying potential interested groups of a campaign -- Product-based strategy

 #### When trying to launch a product or campaign, banks can also identify who are the potential audience based on their past purchases, responsiveness to campaigns and recent views. As such, banks can deliver advertisements to its target audience accurately to achieve the largest return.